Used code provided by Bastian Alexander Grossenbacher for the Machine Learning Lecture at UNIFR

In [ ]:
import pathlib as Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [16]:
project_root = Path().resolve().parents[0]
train_data_path = project_root / "data" / "fashion_mnist" / "train"
test_data_path = project_root / "data" / "fashion_mnist" / "test"

In [19]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

images, _ = next(iter(loader))
mean = images.mean()
std = images.std()

print(f"Mean: {mean.item():.4f}, Std: {std.item():.4f}")

Mean: 0.2860, Std: 0.3530


In [48]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

# Load custom MNIST dataset from folders
dataset = datasets.ImageFolder(
    root=train_data_path,
    transform=transform
)

from torch.utils.data import random_split

# Optional: set seed for reproducibility
generator = torch.Generator().manual_seed(42)

# keep_fraction = 0.3
# keep_size = int(keep_fraction * len(dataset))
# discard_size = len(dataset) - keep_size
# dataset, _ = random_split(dataset, [keep_size, discard_size], generator=generator)


# Set split sizes (adjust ratio as needed)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Split the dataset
train_subset, val_subset = random_split(dataset, [train_size, val_size], generator=generator)
print("Full dataset size:", len(dataset))
print("Train subset size:", len(train_subset))
print("Val subset size:", len(val_subset))

Full dataset size: 60000
Train subset size: 48000
Val subset size: 12000


In [46]:
class FashionCNN(nn.Module):
    def __init__(self, num_channels=1, num_classes=10):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv2d(
                in_channels=num_channels,
                out_channels=32,
                kernel_size=2,
                stride=1,
            ),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=2,
                stride=1,
            ),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),


            nn.Conv2d(
                in_channels=64, out_channels=128, kernel_size=3, stride=1
            ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(1),
            nn.Dropout(),
            nn.LazyLinear(num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

learning_rate = 0.001
batch_size = 128
epochs = 10


train_loader = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle= True)
val_loader = DataLoader(dataset=val_subset, batch_size=batch_size, shuffle= True)

transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((mean), (std)),
        ]
    )

model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()

for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)

            scores = model(data)
            loss = criterion(scores, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = scores.argmax(1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)

        accuracy = 100.0 * correct / total
        print(
            f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%"  # noqa
        )

model.eval()

with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)

            predicted = outputs.argmax(1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        print(f"Test Accuracy: {100. * correct / total:.2f}%")


Epoch [1/10], Loss: 0.4778, Accuracy: 82.83%
Epoch [2/10], Loss: 0.3492, Accuracy: 87.54%
Epoch [3/10], Loss: 0.3085, Accuracy: 88.79%
Epoch [4/10], Loss: 0.2857, Accuracy: 89.64%
Epoch [5/10], Loss: 0.2617, Accuracy: 90.65%
Epoch [6/10], Loss: 0.2426, Accuracy: 91.01%
Epoch [7/10], Loss: 0.2318, Accuracy: 91.53%
Epoch [8/10], Loss: 0.2165, Accuracy: 92.08%
Epoch [9/10], Loss: 0.2109, Accuracy: 92.31%
Epoch [10/10], Loss: 0.1971, Accuracy: 92.84%
Test Accuracy: 91.91%
